In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers, models, applications
from keras.datasets import cifar10

In [ ]:
print('Number of GPUs Available', len(tf.config.list_physical_devices('GPU')))
print("TensorFlow Version:", tf.__version__)

In [2]:
# Random seeds
tf.random.set_seed(42)

# Pre process dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalization
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [18]:
def create_mobilenet_model(input_shape=(32, 32, 3), num_classes=10):
    base_model = applications.MobileNetV2(input_shape=input_shape, include_top=False, weights=None)

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

model = create_mobilenet_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [19]:
#Data Augmentation for Generalization

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('Horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

In [ ]:
# Model Training

history = model.fit(
    data_augmentation(x_train), y_train,
    batch_size = 128,
    epochs = 20,
    validation_data = (x_test, y_test),
    verbose = 1
)

Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 85s 183ms/step - accuracy: 0.2058 - loss: 2.1048 - val_accuracy: 0.1000 - val_loss: 2.3040
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 73s 187ms/step - accuracy: 0.2961 - loss: 1.8542 - val_accuracy: 0.1000 - val_loss: 2.3096
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 72s 184ms/step - accuracy: 0.3403 - loss: 1.7506 - val_accuracy: 0.1000 - val_loss: 2.3125
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 73s 186ms/step - accuracy: 0.3816 - loss: 1.6782 - val_accuracy: 0.1000 - val_loss: 2.3096
Epoch 5/20
 15/391 ━━━━━━━━━━━━━━━━━━━━ 1:08 181ms/step - accuracy: 0.3984 - loss: 1.5960

In [ ]:
# Model Evaluation

test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 0)
print(f"\nTest  accuracy: {test_acc:.4f}")

In [ ]:
# History of Model Training/Evaluation

plt.plot(history.history['accuracy'], label = 'train_accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()